# Chapter 13. Operator Overloading: Doing It Right
- How Python supports infix operators with operands of different types
- Using duck typing or explicit type checks to deal with operands of various types
- How an infix operator method should signal it cannot handle an operand
- The spcial behavior of the rich comparison operators (e.g, ==, >, <=, etc.)
- The default handling of augmented assignment operators, like +=, and how to overload them

# Operator Overloading 101
- Limitations to prevent from abuse
    - We cannot overload operators for the built-in types.
    - We cannot create new operators, only overload existing ones.
    - A few operators can't be overload:
        - `is`, `and`, `or`, `not` 
        - but `&`, `|`, `-`, can

# Unary Operators
- `-(__neg__)`
- `+(__pos__)`
    - `x == +x` is not equal in a few cases.
- `~(__invert__)`
    - bitwise inverse. e.g; if x is 2 then `-x == -3`.

## e.g, `__add__` Operation Flow
1. If `a` has `__add__`, call `a.__add__(b)` and return result unless it's `NotImplemented`.
1. If `a` doesn't have `__add__`, or calling it returns `NotImplemented`, check if `b` has `__radd__`, then call `b.__radd__(a)` and return result unless it's `NotImplemented`.
1. If `b` doesn't have `__radd__`, or calling it returns `NotImplemented`, raise `TypeError` with an `unsupported operand types` message.

<br>

- For duck typing, maybe better to refrain from testing the type of operands.
    - When the eval is failed, need to catch the exceptions and return `Notimpelemented`.

In [16]:
from array import array
import reprlib
import math
import functools
import operator
import itertools
import numbers


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))

    def __eq__(self, other):
        # Below code is inefficient for large length instances
        # return tuple(self) == tuple(other)

        # Efficient example 1
        # if len(self) != len(other):
        #     return False
        # for a, b in zip(self, other):
        #     if a != b:
        #         return False
        # return True

        # Efficinet example 2
        # return len(self) == len(other) and all(a == b for a, b in zip(self, other))
        
        if isinstance(other, Vector):
            return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented

    def __hash__(self):
        # hashes = (hash(x) for x in self._components)
        hashes = map(hash, self._components)
        # map method is lazy because it creates a generator to save memory.
        # This is the same as genexp
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self):
        return Vector(self)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, scalar):
        if isinstance(scalar, numbers.Real):
            return Vector(n * scalar for n in self)
        else:
            return NotImplemented
    
    def __rmul__(self, scalar):
        return self * scalar

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self.components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n - 1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

In [4]:
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [7]:
v1 + Vector((1, 2))

Vector([4.0, 6.0, 5.0])

In [9]:
v1 = Vector([1.0, 2.0, 3.0])
14 * v1

Vector([14.0, 28.0, 42.0])

In [10]:
v1 * True

Vector([1.0, 2.0, 3.0])

In [12]:
from fractions import Fraction
v1 * Fraction(1, 3)

Vector([0.3333333333333333, 0.6666666666666666, 1.0])

In [15]:
t1 = (0, 1)
t2 = (2, 3)
t1 @ t2

TypeError: unsupported operand type(s) for @: 'tuple' and 'tuple'

# Rich Comparison Operators
The Zen of Python says:
> In the face of ambiguity, refuse the temptation to guess.

Excessive liberality in the evaluation of operands may lead to surprising results, and programmers hate surprises.

In [18]:
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
va == vb

True

In [19]:
va == (1.0, 2.0, 3.0)

False

# Augmented Assignment Operators

In [21]:
import abc

class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""
    
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        
        This method should raise `LookupError` when the instance is empty.
        """
    
    def loaded(self):
        """Return `True` if there's at least 1 item. `False` otherwise."""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [23]:
import random

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items = items
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [ ]:
import itertools

class AddableBingoCage(BingoCage):
    def __add__(self, other):
        if isinstance(other, Tombola):
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other)
            except TypeError:
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or an iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)
        return self